In [2]:
import os
import argparse
from bertopic import BERTopic
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

# TODO  check babyplot for 3D visualisation

/Users/kiliansprenkamp/miniconda3/envs/refugeeAnalysis/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# testing

In [ ]:
def read_data(input_file):
    with open(input_file) as file:
        lines = file.readlines()
        text_to_analyse_list = [line.rstrip() for line in lines]
    return text_to_analyse_list

In [ ]:
data = read_data("../data/BERTopicInput.csv")

In [ ]:
model = BERTopic(verbose=True, language="multilingual", nr_topics=30) # vectorizer_model=vectorizer_model)#, nr_topics=20)
topics, probs = model.fit_transform(data)

In [ ]:
import sys
import itertools
# setting path
sys.path.append('../src/BERTopic/')
from runBERTopic import BERTopicAnalysis

nr_topics_list = ["auto"]
min_topic_size_list = [10,25,50,100,200,400,800,1000,1500]
hbscan_min_cluster_size_list = [25,50,100,200]


for i in itertools.product(nr_topics_list,min_topic_size_list,hbscan_min_cluster_size_list):
    if os.path.exists(f"../models/BERTopic50Char{i[0].capitalize()}Classes{i[1]}MinTopicSize{i[2]}HbscanMinClusterSize/BERTopicmodel"):
        print('wuhu', i)
        continue
    BERTopic_Analysis = BERTopicAnalysis(input_file="../data/BERTopicInput50CharMin.csv",
                                         k_cluster=i[0],
                                         min_topic_size=i[1],
                                         hbscan_min_cluster_size=i[2]
                                     )
    print(i)    
    BERTopic_Analysis.run_all()


In [ ]:
with open("../data/BERTopicInput50CharMin.csv") as file:
    lines = file.readlines()
    text_to_analyse_list = [line.rstrip() for line in lines]

In [ ]:
df = pd.read_csv("../models/BERTopic_50Char10Classes/pred_on_whole_data_translated_Sentiment.csv")

In [ ]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
fp = open("test.txt")
data = fp.read()
tokenizer.tokenize(data)

In [ ]:
def inference(x):
    if len(x) >= 500:
        for i in tokenize.sent_tokenize(x):
            print(i)

In [ ]:
df_value_counts = df[df.chat.isin(['https://t.me/zh_helps_ukraine', 'https://t.me/zh_helps_UArefugee',
       'https://t.me/zurich_hb_help', 'https://t.me/zh_housing',
       'https://t.me/Zurich_UA', 'https://t.me/zh_helps_UArefugees', 'https://t.me/Zh_helps_UA_mums', 'https://t.me/job_sw_ukrainians',
       'https://t.me/zh_back_ukraine', 'https://t.me/zh_helps_logistics',])].value_counts(['cluster', 'week']).reset_index()
df_value_counts.columns = ['cluster', 'week', 'count']
mydict = {'no_class':-1,
          'medical': 0, 
          'teaching': 1,
          'banking': 2,
          'transport_UKR_CH': 3,
          'pets': 4,
          'immigration': 5,
          'transport_train_EU': 6,
          'website_links': 7,
          'volunteering': 8,
          'transport_train_CH': 9}
df_value_counts['cluster_names'] = df_value_counts['cluster'].apply(lambda x: list(mydict.keys())[list(mydict.values()).index(x)])

In [ ]:
df_actors = pd.DataFrame(columns=["cluster_name", "message_sender", "nr_post"])
for i in df.cluster_name.unique():
    df[df['cluster_name']==i].value_counts('messageSender').iloc[:10]
    df_actors_i = df[df['cluster_name']==i].value_counts('messageSender').iloc[:10].reset_index()
    df_actors_i["cluster_name"] = i
    df_actors_i = df_actors_i[["cluster_name", "messageSender", 0]]
    df_actors_i.columns = ["cluster_name", "message_sender", "nr_post"]
    df_actors = pd.concat([df_actors, df_actors_i], axis=0)

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("../models/BERTopic100CharMin2500CharMax_10ClassesRedo/df.csv")
df['uniqueIdentifier'] = df.apply(lambda x: x["chat"] + "_" + str(x["messageID"]), axis=1)
df['uniqueIdentifierReply'] = df.apply(lambda x: x["chat"] + "_" + str(x["messageReplyID"])[:-2], axis=1)
df = df[~df.chat.isin(["https://t.me/campax_ukraine_help_switzerland", "https://t.me/helppetsfromukraine"])] 
mydict = {'no_class':-1,
          'medical': 0, 
          'legal_issues': 1,
          'travel': 2,
          'immigration': 3,
          'childcare': 4,
          'education': 5,
          'veterinarian': 6,
          'banking': 7,
          'passport_information': 8,
          'weblinks': 9}
df['cluster_names'] = df['cluster'].apply(lambda x: list(mydict.keys())[list(mydict.values()).index(x)])
df['week']=df['messageDatetime'].apply(lambda x: pd.to_datetime(x).isocalendar()[1])
df_value_counts = df.groupby(['cluster','week']).count().reset_index()

In [ ]:
import plotly.express as px
# for i in df_value_counts.cluster.unique()[1:]:
fig = px.line(df_value_counts[df_value_counts.cluster_names != "no_class"].sort_values(['week']), x="week", y="count", color='cluster_names', title='Cluster over time')
fig.show()
# fig.write_html("../models/BERTopic_50Char10Classes/AllGroups.html")

In [ ]:
import plotly.express as px
# for i in df_value_counts.cluster.unique()[1:]:
fig = px.line(df_value_counts[df_value_counts.cluster != -1].sort_values(['week']), x="week", y="count", color='cluster_names', title='Cluster over time')
fig.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,10))
for i in df_value_counts.cluster.unique()[1:]:
    plt.plot(df_value_counts[df_value_counts.cluster == i].sort_values(['week'])['week'], df_value_counts[df_value_counts.cluster == i].sort_values(['week'])['count'], linestyle = 'dotted', label=list(mydict.keys())[list(mydict.values()).index(i)])
plt.legend(loc="upper left")
plt.show()

# Sentiment

In [ ]:
df = pd.read_csv("../models/BERTopic_50Char10Classes/pred_on_whole_data_translated.csv")
print(df.shape)
df = df[df.messageTextEnglish.apply(type) == str]
print(df.shape)
df = df[df['messageTextEnglish'].notnull()]
print(df.shape)
df[df["messageText"].str.len()<512]
df = df[df['messageTextEnglish'].str.len() <= 513]
df.reset_index(inplace=True)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment").to("mps")

In [ ]:
labels = ['Negative', 'Neutral', 'Positive']
def analyse_sentiment(x):
    encoded_tweet = tokenizer(x, return_tensors='pt')
    output = model(**encoded_tweet)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return scores
def scores_to_sentiment(x):
    max_value = max(x.tolist())
    max_index = x.tolist().index(max_value)
    return labels[max_index] 
    

In [ ]:
from tqdm import tqdm
tqdm.pandas()

# df['sentiment'] = df['messageTextEnglish'].progress_apply(lambda x: analyse_sentiment(x))
df['sentiment_name'] = df['sentiment'].progress_apply(lambda x: scores_to_sentiment(x))

In [ ]:
import matplotlib.pyplot as plt

# Pie chart, where the slices will be ordered and plotted counter-clockwise:
labels = 'Frogs', 'Hogs', 'Dogs', 'Logs'
sizes = [15, 30, 45, 10]
explode = (0, 0.1, 0, 0)  # only "explode" the 2nd slice (i.e. 'Hogs')

fig1, ax1 = plt.subplots()
ax1.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

In [ ]:
# df_test.columns = ['cluster_name', 'sentiment_name', 'amount']
# df_test['percentage'] = ''
for index, row in df_test.iterrows():
    percentage = row['amount']/df_test[df_test.cluster_name==row['cluster_name']].amount.sum()
    df_test.loc[index, "percentage"] = percentage
    # df_test[df_test.cluster_name==i].amount 
    # series = df_test[df_test.cluster_name==i]['amount'].apply(lambda x: float(x)/float(df_test[df_test.cluster_name==i].amount.sum()))
    

In [ ]:
import plotly.express as px

long_df = px.data.medals_long()

fig = px.bar(df_test, x="cluster_name", y=0, color="sentiment_name", title="Sentiment within cluster")
fig.show()

In [ ]:
import plotly.express as px

long_df = px.data.medals_long()

fig = px.bar(df_test, x="cluster_name", y="percentage", color="sentiment_name", title="Sentiment within cluster")
fig.show()

In [ ]:
for i in df.cluster.unique():
    print(i)
    fig1, ax1 = plt.subplots()
    ax1.pie(df[df.cluster==i].value_counts('sentiment_name').values, labels=df[df.cluster==i].value_counts('sentiment_name').index, autopct='%1.1f%%',
        shadow=True, startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    title = list(mydict.keys())[list(mydict.values()).index(i)]
    fig1.suptitle(title, fontsize=12)

    

In [ ]:
import plotly.express as px

long_df = px.data.medals_long()

fig = px.bar(long_df, x="nation", y="count", color="medal", title="Long-Form Input")
fig.show()

In [ ]:
df = pd.read_csv('/Users/kiliansprenkamp/Desktop/OneDrive - Universität Zürich UZH/migrantZurich/unsupervised-migrant-telegram-analysis/models/BERTopic_50Char10Classes/pred_on_whole_data_translated_sentiment_network.csv')

In [ ]:
df.messageText.str.len().nlargest(3000)

# potential publication

In [45]:
df = pd.read_csv("../models//BERTopic100CharMin2500CharMax_10ClassesNewestData/df.csv")
df['uniqueIdentifier'] = df.apply(lambda x: x["chat"] + "_" + str(x["messageID"]), axis=1)
df['uniqueIdentifierReply'] = df.apply(lambda x: x["chat"] + "_" + str(x["messageReplyID"])[:-2], axis=1)
# df = df[~df.chat.isin(["https://t.me/campax_ukraine_help_switzerland", "https://t.me/helppetsfromukraine"])] 
class_name_dict = {'no_class':-1,
          'medical': 0, 
          'complaints': 1,
          'asylum_volunteering': 2,
          'banking': 3,
          'train_travel': 4,
          'S_Status': 5,
          'accomodation': 6,
          'veterinarian': 7,
          'passport_information': 8,
          'transport_UKR_CH': 9}
df['cluster_names'] = df['cluster'].apply(lambda x: list(class_name_dict.keys())[list(class_name_dict.values()).index(x)])
df['week']=df['messageDatetime'].apply(lambda x: pd.to_datetime(x).isocalendar()[1])
messages_per_week_dict = dict(df.value_counts("week"))
df_value_counts = df.value_counts(["cluster_names", "week"]).reset_index()
df_value_counts.columns = ["cluster_names", "week", "occurence_count"]
df_value_counts['occurence_count_norm'] = df_value_counts.apply(lambda x: x.occurence_count/list(messages_per_week_dict.values())[list(messages_per_week_dict.keys()).index(x.week)], axis=1)

In [46]:
import plotly.express as px
fig = px.line(df_value_counts[df_value_counts.cluster_names != "no_class"].sort_values(['week']), x="week", y="occurence_count_norm", color='cluster_names', title='Cluster over time')
fig.show()

## model after merge topic

In [ ]:
model=BERTopic.load("../models/BERTopic100CharMin2500CharMax_10Classes/merge_topics/BERTopicmodel")

In [ ]:
topics, probabilities = model.fit_transform(docs[:10000])

In [ ]:
model.get_representative_docs().keys()

In [ ]:
writer = pd.ExcelWriter(f"../models/BERTopic100CharMin2500CharMax_10Classes/merge_topics/representative_docs.xlsx", engine='xlsxwriter')
for i in model.get_representative_docs().keys():
    print(i)
    df = pd.DataFrame(model.get_representative_docs()[i], columns=['message'])
    df.to_excel(writer, sheet_name=model.get_topic_info()[model.get_topic_info()['Topic']==i]['Name'].values[0][:31])
writer.save()
model.get_topic_info().to_csv(f"../models/BERTopic100CharMin2500CharMax_10Classes/merge_topics/topic_info.csv")

## model20

In [ ]:
model2=BERTopic.load("../models/BERTopic100CharMin2500CharMax_20Classes/BERTopicmodel")
with open('../data/BERTopicInput100CharMin2500CharMax.csv') as file:
    lines = file.readlines()
    docs = [line.rstrip() for line in lines]

In [ ]:
topics2, probabilities2 = model2.fit_transform(docs)

In [ ]:
import pickle
with open("../models/BERTopic100CharMin2500CharMax_20Classes/probabilities.pkl",'wb') as f:
    pickle.dump(probabilities2, f)
with open("../models/BERTopic100CharMin2500CharMax_20Classes/topics.pkl",'wb') as f:
    pickle.dump(topics2, f)

In [ ]:
with open("../models/BERTopic100CharMin2500CharMax_20Classes/topics.pkl",'rb') as f:
    x = pickle.load(f)


# Multimodal 

In [ ]:
import os
import glob
import zipfile
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from sentence_transformers import SentenceTransformer, util

# Flickr 8k images
img_folder = 'photos/'
caps_folder = 'captions/'
if not os.path.exists(img_folder) or len(os.listdir(img_folder)) == 0:
    os.makedirs(img_folder, exist_ok=True)

    if not os.path.exists('Flickr8k_Dataset.zip'):   #Download dataset if does not exist
        util.http_get('https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip', 'Flickr8k_Dataset.zip')
        util.http_get('https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip', 'Flickr8k_text.zip')

    for folder, file in [(img_folder, 'Flickr8k_Dataset.zip'), (caps_folder, 'Flickr8k_text.zip')]:
        with zipfile.ZipFile(file, 'r') as zf:
            for member in tqdm(zf.infolist(), desc='Extracting'):
                zf.extract(member, folder)
images = list(glob.glob('photos/Flicker8k_Dataset/*.jpg'))

# Prepare dataframe
captions = pd.read_csv("captions/Flickr8k.lemma.token.txt",sep='\t',names=["img_id","img_caption"])
captions.img_id = captions.apply(lambda row: "photos/Flicker8k_Dataset/" + row.img_id.split(".jpg")[0] + ".jpg", 1)
captions = captions.groupby(["img_id"])["img_caption"].apply(','.join).reset_index()
captions = pd.merge(captions, pd.Series(images, name="img_id"), on="img_id")

# Extract images together with their documents/captions
images = captions.img_id.to_list()
docs = captions.img_caption.to_list()


In [ ]:
print("wuhu")
model = SentenceTransformer('clip-ViT-B-32')
print("wuhu")
# Prepare images
batch_size = 32
print("wuhu")
nr_iterations = int(np.ceil(len(images) / batch_size))
print("wuhu")
# Embed images per batch
embeddings = []
for i in tqdm(range(nr_iterations)):
    start_index = i * batch_size
    end_index = (i * batch_size) + batch_size

    images_to_embed = [Image.open(filepath) for filepath in images[start_index:end_index]]
    img_emb = model.encode(images_to_embed, show_progress_bar=False)
    embeddings.extend(img_emb.tolist())

    # Close images
    for image in images_to_embed:
        image.close()
embeddings = np.array(embeddings)

In [ ]:
from bertopic import BERTopic
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words="english")
topic_model = BERTopic(vectorizer_model=vectorizer_model)
topics, probs = topic_model.fit_transform(docs, embeddings[:123])
captions["Topic"] = topics